![Microsoft](https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true)  

# Call from On-Prem SQL Server to Pre-Built Azure AI

## Text Analysis of Database records using Azure Cognitive Services Calls in Python

## Install Python Libraries

This example depends on the Cogntitive Services Python library from Microsoft. In this case we are using the "Big Data Clusters" feature from SQL Server 2019, which runs on a Kubernetes Cluster. 

Using the `kubectl` command, the library is installed using the `pip` python utility:

Connect to the Master instance of SQL Server in the Cluster: 

`kubectl exec --stdin --tty master-0 -- /bin/bash`

Locate the proper directory: 

`cd /opt/mssql/mlservices/bin/python`

Install the proper library using `pip` within the proper Python distribution for Machine Learning Services:

`sudo ./python -m pip install --upgrade azure-ai-textanalytics`

> Note that the `mlutils` utility can also be used for SQL Server Machine Learning Services package management.

In [1]:
/* Get data from the table to use in our Sentiment Analysis */
USE Analysis;
GO

SELECT pr_review_content
FROM dbo.product_reviews
WHERE pr_review_sk = 72621;
GO

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.799

pr_review_content
"Works fine. Easy to install. Some reviews talk about not fitting wall plates. Designed for the best, while greet dinner guests, smelling stronger than the Vollarth. While the handle's grip is nice on the OXO Good Grips Trigger Ice Cream Scoop purchased recently and this is the same as all the difference in the kitchen. If you cook for living, go for the professional series."


In [34]:
EXECUTE sp_execute_external_script @language = N'Python'
    , @input_data_1 = N'SELECT pr_review_content FROM dbo.product_reviews WHERE pr_review_sk = 72621'
    , @input_data_1_name = N'inputdata'
    , @script = N'
key = "8675309"
endpoint = "http://BR549"
print("Keys set")

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import json
import pandas as pd
print("Imports successful")

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, credential=ta_credential)
    return text_analytics_client
print("Client set")

client = authenticate_client()
print("Client authenticated")

documents = inputdata["pr_review_content"].tolist()

def sentiment_analysis_example(client):
    response = client.analyze_sentiment(documents = documents)[0]
    print("Document Sentiment: {}".format(response.sentiment))
    print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f} \n".format(
        response.confidence_scores.positive,
        response.confidence_scores.neutral,
        response.confidence_scores.negative,
    ))
    for idx, sentence in enumerate(response.sentences):
        print("Sentence: {}".format(sentence.text))
        print("Sentence {} sentiment: {}".format(idx+1, sentence.sentiment))
        print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
            sentence.confidence_scores.positive,
            sentence.confidence_scores.neutral,
            sentence.confidence_scores.negative,
        ))
print("Function set")

sentiment_analysis_example(client)
print("Analysis Complete")
'

STDOUT message(s) from external script: 
Keys set
Imports successful
Client set
Client authenticated
Function set
Document Sentiment: mixed
Overall scores: positive=0.76; neutral=0.12; negative=0.12 

Sentence: Works fine.
Sentence 1 sentiment: positive
Sentence score:
Positive=1.00
Neutral=0.00
Negative=0.00

Sentence: Easy to install.
Sentence 2 sentiment: positive
Sentence score:
Positive=1.00
Neutral=0.00
Negative=0.00

Sentence: Some reviews talk about not fitting wall plates.
Sentence 3 sentiment: negative
Sentence score:
Positive=0.01
Neutral=0.27
Negative=0.72

Sentence: Designed for the best, while greet dinner guests, smelling stronger than the Vollarth.
Sentence 4 sentiment: positive
Sentence score:
Positive=0.96
Neutral=0.04
Negative=0.00

Sentence: While the handle's grip is nice on the OXO Good Grips Trigger Ice Cream Scoop purchased recently and this is the same as all the difference in the kitchen.
Sentence 5 sentiment: positive
Sentence score:
Positive=0.99
Neutral=0.01

STDOUT message(s) from external script: 
Negative=0.00

Sentence: If you cook for living, go for the professional series.
Sentence 6 sentiment: positive
Sentence score:
Positive=0.57
Neutral=0.42
Negative=0.01

Analysis Complete

Total execution time: 00:00:03.564